# Tableau de bord

In [ ]:
import time
t0 = time.time()
import json
from pathlib import Path
import panel as pn

## Load config:
config_path = Path('config.json')
if not config_path.is_file():
    !python update_config.py
if config_path.is_file():
    with open(config_path,encoding='utf-8') as config_file:
        config = json.load(config_file)
else:
    raise ValueError("config.json doesn't exist!")

## Set CSS:
pn.extension(
    raw_css=[config["css"].replace('\\n','').replace('\n','')],
    loading_spinner='dots',
    loading_color='#00aa41'
)

## First load: just the dashboard, help buttons, and language.
dash = pn.template.VanillaTemplate(title="Climate Analogues | Analogues climatiques")

sidebar = pn.FlexBox(align_content='flex-start',justify_content='flex-start', flex_wrap='nowrap', flex_direction='column', sizing_mode='stretch_both',css_classes=['flex-sidebar'])
main    = pn.FlexBox(align_content='flex-start',justify_content='center', align_items='center',flex_wrap='nowrap', flex_direction='column', sizing_mode='stretch_both')
modal   = pn.FlexBox(align_content='space-evenly',justify_content='space-evenly', flex_wrap='nowrap', flex_direction='column', sizing_mode='stretch_both')
header  = pn.FlexBox(align_content='space-evenly',justify_content='space-evenly', flex_wrap='nowrap', flex_direction='column', sizing_mode='stretch_both')

dash.sidebar.append(sidebar)
dash.main.append(main)
dash.modal.append(modal)
dash.header.append(header)

w_sidetitle = pn.pane.Markdown('##Start new search')
sidebar.append(w_sidetitle)
w_loading_spinner = pn.indicators.LoadingSpinner(height=100,width=100,value=True,color="primary")
w_loading_text = pn.panel("Loading app...",
                   style={'background-color':'var(--primary)','color':'white','border-radius':'25px',"padding-left":"10px","padding-right":"10px"})
w_loading = pn.Column(w_loading_spinner, w_loading_text)

sidebar.append(pn.Row(pn.layout.HSpacer(),w_loading,pn.layout.HSpacer()))
main.append(pn.Column(pn.layout.VSpacer(),w_loading,pn.layout.VSpacer()))

docpath = Path('./docs')
docs = {}
if docpath.is_dir():
    for file in docpath.glob('*.md'):
        with open(file,'r') as f:
            docs[file.stem] = f.read()
## MODAL: 
w_enter_en = pn.widgets.Button(name='Enter')
w_enter_fr = pn.widgets.Button(name='Entrez')


w_about_en = pn.Column(pn.pane.Markdown('## About: Spatial Analogues'), pn.pane.Markdown(docs['info_en'],height=320),w_enter_en)
w_about_fr = pn.Column(pn.pane.Markdown('## À Propos: Analogues Spatiaux'),pn.pane.Markdown(docs['info_fr'],height=320),w_enter_fr)
modal_lang = pn.Row(pn.layout.HSpacer(),w_about_en,pn.layout.HSpacer(),w_about_fr,pn.layout.HSpacer(),min_width=600)
modal.append(modal_lang)

def open_modal(event):
    dash.open_modal()
    
## HEADER:
w_open_modal = pn.widgets.Button(name='About | À Propos', width = 150)
w_open_modal.on_click(open_modal)
pn.state.onload(dash.open_modal)

LOCALE = "en"

w_language = pn.widgets.Button(name="Français", width=150)
w_headerbox = pn.Row(pn.layout.HSpacer(),w_open_modal, w_language)
header.append(w_headerbox)

t1 = time.time()
print('Time to first load:',t1 - t0)



In [15]:



def get_helppage(locale):
    docpages = {'howto':{"en":"How to use this app","fr":"Comment utiliser cette application"},
                'interp':{"en":"Interpreting Results","fr":"Interprétation des résultats"},
                'advanced':{"en":"Advanced Options","fr":"Options avancées"},
                'attribution':{"en":"Attribution and Sources","fr":"Attribution et Sources"}
               }
    docpage_locale = {k+'_'+locale:v[locale] for k,v in docpages.items()}
    markdowns = [pn.pane.Markdown(object=f'<div id="{k}"/>\n'+ docs[k],min_width=800,width=920) for k,v in docpage_locale.items()]
    linkhtml_en = ''.join(["<h1>Help</h1><h2>Contents:</h2><table class='link-table'>",*[f'<tr><td><a href="#{page}">{i+1}– {title}</td></tr>' for i,(page,title) in enumerate(docpage_locale.items())],"</table>"])
    linkhtml_fr = ''.join(["<h1>Aide</h1><h2>Contenu:</h2><table class='link-table'>",*[f'<tr><td><a href="#{page}">{i}. {title}</td></tr>'  for i,(page,title) in enumerate(docpage_locale.items())],"</table>"])

    links = pn.pane.HTML(linkhtml_en,width=500)
    
    helppage = pn.Column(name={"en":"Help","fr":"Aide"}[locale],min_width=800,width=920)
    helppage.append(links)
    [helppage.append(markdown) for markdown in markdowns]
    return helppage

Column(min_width=800, name='Aide', width=920)
    [0] HTML(str, width=500)
    [1] Markdown(str, min_width=800, width=920)
    [2] Markdown(str, min_width=800, width=920)
    [3] Markdown(str, min_width=800, width=920)
    [4] Markdown(str, min_width=800, width=920)

In [ ]:
# panel has difficulty with local modules, it seems.

from core import utils, widgets, search
from core.constants import fut_col, hist_col, ana_col, quality_terms, quality_colors

main.clear()
searches = widgets.TabsMod(get_helppage(LOCALE),closable=True)
searches.closablelist[0] = False
main.append(searches)

In [ ]:
try_again = pn.widgets.Button(name="Try again?",width=300,button_type='danger')

def update_handled(language=LOCALE):
    try:
        return update_dashboard(language)
    except Exception as e:
        # change to "app not available", change color.
        w_loading.clear()
        error_text=pn.panel("Error loading app...",
                           style={'background-color':'#A00','color':'white','border-radius':'25px',"padding-left":"10px","padding-right":"10px"})
        error_cause=pn.pane.Str("Error Log: "+type(e) + "\n" + str(e))
        
        w_loading.append(pn.FlexBox(error_circle,error_text,error_cause,try_again,flex_direction='column',align_items='center'))
        print(str(e))
        
try_again.on_click(update_handled)

def update_dashboard(language=LOCALE):
    t1 = time.time()
    ''' these modules are heavy to load the first time, defering their import can help.'''
    # Paquets
    from collections import namedtuple
    import dask
    from dask.diagnostics import ProgressBar
    import geopandas as gpd
    import geoviews as gv
    import holoviews as hv
    import hvplot.xarray
    from io import StringIO
    import numpy as np
    import pandas as pd
    from panel.viewable import Viewer
    from bokeh.models import HoverTool
    import param
    from shapely.geometry import Point, LineString
    import xarray as xr
    import xclim as xc
    from xclim import analog as xa
    from datetime import datetime
    import pickle
    import joblib
    import warnings
    from shapely.errors import ShapelyDeprecationWarning

    warnings.filterwarnings("ignore",category=ShapelyDeprecationWarning)
    if not utils.check_version(config):
        print("Old config version detected. Removing cached files.")
        !rm density.obj
        !rm benchmark.obj
        !rm -rf ./cache/*
    
    t2 = time.time()
    print("time to import:", t2 - t1)
    
    global cities, dref, dsim, biasadjust, init_rand_city, benchmark, density

    gv.extension('bokeh')

    # constants
    # Dask. To make this dashboard slightly faster, change the "scheduler" argument to scheduler='processes' and num_workers=4 (for example)
    # However the final webapp most likely won't have access to this kind of parallelism
    dask.config.set(scheduler=config["options"]["dask_schedule"], temporary_directory='/notebook_dir/writable-workspace/tmp')
    try:
        curr_dir = Path(__file__).parent
    except NameError:  # When running as a notebook "__file__" isn't defined.
        curr_dir = Path('.')
    cities_file = f'{curr_dir}/cities_tmp.geojson'

    # Projection
    biasadjust = config["options"]["biasadjust"] # scaling or dqm the method used for the annual adjustment method

    # Random city on load
    init_rand_city = config["options"]["init_rand_city"]

    # Data
    dref = utils.open_thredds(
        config["url"]["dref"]
    ).chunk({'time': -1}).sel(time=slice('1991', '2020'))

    dsim = utils.open_thredds(config["url"]["dsim"])
    dsim = xc.core.calendar.convert_calendar(dsim, 'default')

    places = gpd.read_file(config["url"]["places"]).to_crs(epsg=8858)
    
    cities = gpd.read_file(cities_file)

    # Pre-compute/download the reference distributions and the density map
    if not Path('benchmark.obj').is_file():
        benchmark = utils.open_thredds(config["url"]["benchmark"]).benchmark.load()

        with open('benchmark.obj', 'wb') as obj_handler:
            pickle.dump(benchmark, obj_handler)
    else:
        with open('benchmark.obj', 'rb') as obj_handler:
            benchmark = pickle.load(obj_handler)

    if not Path('density.obj').is_file():
        masks = utils.open_thredds(config["url"]["masks"])

        density = masks.dens_adj.sel(year=2020).where(
            masks.roi & dref.isel(time=0).notnull().to_array().all('variable')
        ).load()

        with open('density.obj', 'wb') as obj_handler:
            pickle.dump(density, obj_handler)

    # load pickled data
    else:
        with open('density.obj', 'rb') as obj_handler:
            density = pickle.load(obj_handler)
    
    
    t3 = time.time()
    print("Time to load objs:",t3 - t2)
    
    CartoLabels = gv.element.WMTS('https://a.basemaps.cartocdn.com/light_only_labels/{z}/{x}/{y}@2x.png', name='CartoLabels')
    CartoBase = gv.element.WMTS('https://cartodb-basemaps-4.global.ssl.fastly.net/light_nolabels/{Z}/{X}/{Y}@2x.png', name="CartoBase")

    CDNLabelsEn = gv.element.WMTS('https://maps-cartes.services.geo.ca/server2_serveur2/rest/services/BaseMaps/CBMT_TXT_3857/MapServer/WMTS/tile/1.0.0/BaseMaps_CBMT_TXT_3857/default/default/{z}/{y}/{x}.png', name='CDNLabelsEn')
    CDNLabelsFr = gv.element.WMTS('https://maps-cartes.services.geo.ca/server2_serveur2/rest/services/BaseMaps/CBCT_TXT_3857/MapServer/WMTS/tile/1.0.0/BaseMaps_CBMT_TXT_3857/default/default/{z}/{y}/{x}.png', name='CDNLabelsFr')

    LabelMap = CartoLabels if (language == "en") else CartoLabels
    
    # CartoBase = WMTS('https://b.basemaps.cartocdn.com/rastertiles/voyager_nolabels/{Z}/{X}/{Y}@2x.png', name="CartoBase")
    EsriTopo = gv.element.WMTS('https://server.arcgisonline.com/ArcGIS/rest/services/World_Physical_Map/MapServer/tile/{Z}/{Y}/{X}@2x', name="EsriTopo").opts(alpha=0.5, max_zoom=8)
    # CartoLabels = WMTS('https://b.basemaps.cartocdn.com/rastertiles/voyager_only_labels/{Z}/{X}/{Y}@2x.png',name='CartoLabels')
    w_city = pn.widgets.Select(
        name={"en":'Target city',"fr":"Ville cible"}[language],
        options={f"{city.province}: {city.city}": i for i, city in cities.iterrows()},
        width=300,min_width=300,max_width=300
    )
    #if init_rand_city:
    #    def random_city():
    #        w_city.value = np.random.randint(0, len(cities))
    #
    #    pn.state.onload(random_city)
        
    w_ssp = pn.widgets.RadioButtonGroup(
                    options={{"en":f"{y} ({x})","fr":f"{y} ({x})"}[language]:x 
                               for y,x in zip({"en":["Moderate","High"], "fr":["Modérés","Élevés"]}[language],dsim.ssp.values)},
                  sizing_mode='stretch_width',width_policy='max')
    w_ssp_labelled = pn.Column({"en":'Emissions scenario:',"fr":"Scénario d'émissions :"}[language],w_ssp, width=300,min_width=300,max_width=300)
                  
    
    
    w_tgt_period = pn.widgets.DiscreteSlider(
        name={"en":'Target period',"fr":"Période ciblé"}[language],
        options={{"en":f"{x-29}-{x}","fr":f"{x-29} à {x}"}[language]: slice(f"{x-29}", f"{x}") for x in range(2020, 2101, 10)},
        value=slice("2041", "2070"), width=300,min_width=300,max_width=300
    )

    w_indices = pn.widgets.MultiChoice(
        name={"en":'Climate indices',"fr":"Indices climatiques"}[language],
        max_items=4,
        options=[], width=300,min_width=300,max_width=300
    )

    @pn.depends(icity=w_city, ssp=w_ssp, tgt_period=w_tgt_period)
    def usable_indices(icity, ssp, tgt_period):
        with pn.param.set_values(w_indices, loading=True):
            unusable = search.get_unusable_indices(cities, dref, dsim, icity, ssp, tgt_period)
            options = {{"en":v.long_name,"fr":v.long_name_fr}[language]: k for k, v in dsim.data_vars.items() if k not in unusable}
            values = [v for v in w_indices.value if v not in unusable]
            w_indices.options = options
            w_indices.value = values
        #if unusable:
        #    return pn.pane.Alert(
        #        "Some indices are not usable for this combination of city, scenario and target period.",
        #        alert_type='warning'
        #    )
        return pn.pane.Str('',visible=False)


    w_density_factor = pn.widgets.IntSlider(name={"en":'Density range factor',"fr":"Facteur de densité"}[language], value=4, step=1, start=2, end=10,width=280)

    @pn.depends(icity=w_city, density_factor=w_density_factor)
    def info(icity, density_factor):
        dens = cities.iloc[icity].density
        dmin = max(dens / density_factor, 10)
        dmax = dens * density_factor
        N = ((density < dmax) & (density > dmin)).sum().item()
        return pn.pane.Markdown(
            {"en": f"* Target population density : {dens:.0f} people per km²\n"
                    f"* Population density range : {dmin:.0f} - {dmax:.0f} people per km²\n"
                    f"* Number of search candidates : {N}",
             "fr":f"* Densité de la ville cible : {dens:.0f} hab./ km²\n"
                    f"* Densités admissibles : {dmin:.0f} - {dmax:.0f} hab. / km²\n"
                    f"* Nombre de candidats de recherche : {N}"
            }[language],width=280
        )
    w_show_poor = pn.widgets.Checkbox(name={"en":"Display poor quality analogues","fr":"Montrer les analogues de mauvaise qualité"}[language],value=False,width=280)
    w_run = pn.widgets.Button(name="",min_width=300, max_width=300,width=300)

    w_progress = pn.widgets.Progress(active=False, min_width=300, width=300) # Progress(active=False, delta=0.1, min_width=200, width=300)

    @pn.depends(indices = w_indices)
    def enable_search(indices):
        if len(indices) >= 1:
            w_run.disabled = False
            w_run.name = {"en":"Run analogues search","fr":"Nouvelle recherche"}[language]
        else:
            w_run.disabled = True
            w_run.name = {"en":"Select some climate indices!","fr":"Selectionnez des indices !"}[language]
        return pn.pane.Str('',visible=False)
    #@pn.depends(clicks=w_run.param.clicks)
    def analogs_search(clicks):
        """This function does everything."""
        if clicks == 0:
            return pn.pane.Str({"en":'Please run an analogue search using the sidebar.',"fr":"Faites une nouvelle recherche avec la barre de gauche."}[language])

        w_progress.active = True

        # Translate the widget's values to variables
        # The goal is to keep the code here and in the notebook in sync so that copy-pasting the main parts doesn't break
        icity = w_city.value
        ssp = w_ssp.value
        ssp_opts = dsim.ssp.values
        tgt_period = w_tgt_period.value
        periods = list(w_tgt_period.options.values())
        climate_indices = w_indices.value
        density_factor = w_density_factor.value
        max_density = w_density_factor.end
        show_poor = w_show_poor.value

        ### Analogue finding begins here. Code below should be the exact same as in the notebook
        city = cities.iloc[icity]

        #sim = dsim[climate_indices].isel(location=icity).sel(ssp=ssp)

        analogs, sim, ref = search.analogs(dsim, 
                                                  dref, 
                                                  density, 
                                                  benchmark, 
                                                  city,cities,places, 
                                                  climate_indices, 
                                                  density_factor,max_density, 
                                                  tgt_period,periods, 
                                                  ssp,ssp_opts)
        if not show_poor:
            filter_rows = np.where(analogs['qflag'] > 2)[0]
            if len(filter_rows) < analogs.shape[0]:
                analogs.drop(filter_rows, inplace = True)
        
        selector = widgets.ColoredToggleGroup(analogs.quality)

        # Map of analogues
        @pn.depends(iana=selector.param.value)
        def chosen_point(iana):
            return gv.Points([analogs.iloc[iana].geometry])

        analogs_lines = gpd.GeoDataFrame(
            analogs.drop(columns=['geometry']),
            geometry=[LineString([city.geometry, geom]) for geom in analogs.geometry]
        )

        shown_dims = {'simulation':'simulation','near':'près de','quality':'qualité', 'rank':'rang'}
        hover = HoverTool(tooltips=[(k if (language == "en") else v, "@"+k) for k,v in shown_dims.items()])
        point_map = gv.Points(analogs).opts(tools=[hover],color='quality', 
                                                              marker='circle',
                                                              size=10, 
                                                              cmap=dict(zip(quality_terms, quality_colors)), 
                                                              line_color='k')

        analog_map =  pn.pane.HoloViews(
            (
                CartoBase
                * EsriTopo
                * LabelMap
                * gv.Points([city.geometry]).opts(color=fut_col, marker='star', size=15)
                * gv.Path(analogs_lines, vdims=['simulation', 'quality', 'rank', 'near'])
                * point_map
                * gv.DynamicMap(chosen_point).opts(color=ana_col, marker='circle', fill_color='none', size=20, line_width=4)
            ).opts(width=600, height=500, title={"en":'Map of analogues',"fr":"Carte d'analogues"}[language]),
            min_width=600
        )

        # Cards

        cards = pn.Accordion(min_width=800,width=920)
        climdict = {}
        for climind in climate_indices:
            climdict[sim[climind].long_name] = climind
            name = sim[climind].long_name
            data = pn.Column(name=name,min_width=800, min_height=900, width=920)
            cards.append(data)

        @pn.depends(show=cards.param.active, iana=selector.param.value)
        def get_card_data(show,iana):
            for i,panelcard in enumerate(cards.objects):
                if i not in show:
                    panelcard.visible = False
                    #panelcard.min_height=0
                    #panelcard.height = 0
            if not show:
                return cards
            else:
                for panelcardind in show:
                    panelcard = cards.objects[panelcardind]
                    panelcard.visible = True
                    #panelcard.min_height = 900
                    #panelcard.height = 900

                    computation_needed = not utils.is_computed(ref)
                    if computation_needed:
                        panelcard.clear()
                        panelcard.insert(0,pn.pane.Markdown({"en":"### Computing univariate statistics...","fr":"### Calcule de statistiques univariés..."}[language],min_width=500))
                        w_progress.active = True
                        utils.inplace_compute(ref)
                        w_progress.active = False

                    analog = analogs.iloc[iana]

                    climind = climdict[panelcard.name]
                    refi = ref[climind].sel(site=analog.site)
                    histi = sim[climind].sel(realization=analog.simulation, time=slice('1991', '2020'))
                    simi = sim[climind].sel(realization=analog.simulation, time=tgt_period)

                    vmin = min(histi.mean() - 3 * histi.std(), refi.min(), simi.min())
                    vmax = max(histi.mean() + 3 * histi.std(), refi.max(), simi.max(), 2 * histi.mean() - vmin)
                    vmin = 2 * histi.mean() - vmax
                    xlim = (float(vmin), float(vmax))

                    uni_score = xa.zech_aslan(simi, refi)
                    qflag = utils.get_quality_flag(uni_score, [climind], benchmark)

                    units = f"[{simi.units}]"
                    name = {"en":simi.long_name,"fr":simi.long_name_fr}[language]

                    dist_diff = (
                        hv.Distribution(simi.values, label={"en":"Target's future","fr":"Ville ciblé dans le futur"}[language]).opts(color=fut_col)
                        * hv.Distribution(refi.values, label={"en":"Analogue's present","fr":"Analogue dans le présent"}[language]).opts(color=ana_col)
                        * hv.Distribution(histi.values, label={"en":"Target's present","fr":"Ville ciblé dans le présent"}[language]).opts(color='white', line_color='black')
                    ).opts(
                        yaxis=None, ylabel='dist', xlabel=name, responsive=True, aspect=3,
                        legend_cols=True, legend_offset=(0, 0), legend_position='bottom', fontscale=0.8,
                        title={"en":'Distribution comparison',"fr":"Comparaison de distribution"}[language], xlim=xlim,
                    )

                    mean_change = (
                        hv.Overlay(
                            [hv.VLine((histi.mean() + n * histi.std()).item()).opts(color='pink', line_dash='dashed', alpha=0.5)
                             for n in range(-3, 4)]
                        ) * hv.Points([[refi.mean().item(), 1]], label="Analogue").opts(color=ana_col, size=20, marker='circle')
                        * hv.Points([[histi.mean().item(), 1]], label={"en":"Target's present","fr":"Ville ciblé dans le présent"}[language]).opts(color=hist_col, size=20, marker='star', line_color='k')
                        * hv.Points([[simi.mean().item(), 1]], label={"en":"Target's future","fr":"Ville ciblé dans le futur"}[language]).opts(color=fut_col, size=20, marker='star')
                    ).opts(
                        yaxis=None, xlim=xlim, responsive=True, height=100, xlabel=name,
                        show_legend=False, # legend_position='bottom', legend_offset=(0, 0), legend_cols=True, 
                        fontscale=0.8, title={"en":'Average change',"fr":"Changement moyenne"}[language], ylabel='nothing'
                    )

                    if int(tgt_period.start) >= 2020:
                        refcp = refi.assign_coords(time=simi.time).hvplot(color=ana_col, hover=False, legend=False)
                    else:
                        refcp = hv.Overlay()

                    timeseries = (
                        (hv.VLine(simi.indexes['time'][0]) * hv.VLine(simi.indexes['time'][-1])).opts(hv.opts.VLine(color='lightblue', line_width=2))
                        * sim[climind].hvplot(by='realization', color='lightgrey', alpha=0.5, hover=False, legend=False)
                        * sim[climind].median('realization').hvplot(color='darkgrey', hover=False, legend=False)
                        * refi.hvplot(color=ana_col, label={"en":'Selected analogue',"fr":"Analogue choisi"}[language])
                        * refcp
                        * sim[climind].sel(realization=analog.simulation).hvplot(color=fut_col, label={"en":'Selected simulation on target city',"fr":"Simulation choisi dans la ville cible"}[language])
                    ).opts(
                        ylabel=units, xlabel='', title={"en":'Full timeseries',"fr":"Série temporelle complète"}[language], legend_position='top',
                        responsive=True, aspect=2, show_legend=False
                    )
                    description = pn.pane.Markdown(
                        {"en":f"### Quality of univariate analogy: {uni_score: 5.2f} ({quality_terms[qflag]})\n"
                        f'- **Description**: {simi.description}\n'
                        f'- **Units** : {simi.units}\n\n',
                         "fr":f"### Qualité de l'analogie univarié: {uni_score: 5.2f} ({quality_terms[qflag]})\n"
                        f'- **Description**: {simi.description_fr}\n'
                        f'- **Unités** : {simi.units}\n\n'
                        }[language],
                        min_width = 500
                    )
                    holoview = pn.pane.HoloViews(hv.Layout([dist_diff, mean_change]).cols(1), linked_axes=False,min_width=800, width=920)

                    panelcard.clear()
                    panelcard.insert(0,description)
                    panelcard.insert(1,holoview)
                    panelcard.insert(2,timeseries)
                return cards

        @pn.depends(iana=selector.param.value)
        def summary(iana):
            analog = analogs.iloc[[iana]].to_crs(epsg=8858)


            data = {
                {"en":'Urban area',"fr":"Ville"}[language]: [f"{city.city}, {city.prov_code}", {"en":"near","fr":"près de"}[language] + f" {analog.iloc[0].near} ({analog.iloc[0].near_dist:.0f} km)"],
                {"en":'Coordinates',"fr":"Coordonées"}[language]: [f"{utils.dec2sexa(city.geometry.y)}N, {utils.dec2sexa(-city.geometry.x)}W",
                                f"{utils.dec2sexa(analogs.iloc[iana].geometry.y)}N, {utils.dec2sexa(-analogs.iloc[iana].geometry.x)}W"],
                {"en":"Time period","fr":"Période de temps"}[language]: [f"{tgt_period.start}-{tgt_period.stop}", "1991-2020"],
                {"en":"Data source","fr":"Source de données"}[language]: [f"{analog.iloc[0].simulation} / SSP{ssp[3]}-{ssp[4]}.{ssp[5]}", "ERA5-Land"],
                {"en":"Pop. density","fr":"Densité urbaine"}[language]: [f"{city.density:.0f} hab/km²", f"{analog.iloc[0].density:.0f} hab/km²"]
            }
            perc_fmt = '.0f' if analog.iloc[0].percentile > 1 else ('.2f' if analog.iloc[0].percentile > 0.01 else '.04f')
            return pn.Column(
                pn.pane.Markdown(
                    {"en":f'### Current selection : \#{iana + 1}\n'
                    f'**Quality of analogy**: {analog.iloc[0].quality} ({analog.iloc[0].score:.3f}, top {analog.iloc[0].percentile:{perc_fmt}} %)\n'
                    f'**Representativeness score**: {analog.iloc[0].zscore:.2f}',
                     "fr":f'### Sélection choisie : \#{iana + 1}\n'
                    f"**Qualité de l'analogie**: {analog.iloc[0].quality} ({analog.iloc[0].score:.3f}, meilleure {analog.iloc[0].percentile:{perc_fmt}} %)\n"
                    f'**Score de représentation**: {analog.iloc[0].zscore:.2f}'
                    }[language]
                ),
                pn.pane.DataFrame(
                    pd.DataFrame.from_dict(data, orient='index', columns=[{"en":'Target',"fr":"Cible"}[language], 'Analogue']),
                    min_width=300
                ),        
            )

        w_progress.active = False
        return pn.FlexBox(
            selector,
            pn.layout.Divider(),
            pn.Row(analog_map, summary),
            get_card_data,
            max_width=1000,
            name=city.city,
            align_content='center',justify_content='flex-start', flex_wrap='nowrap', flex_direction='column', width=920,min_width=800
        )

    advanced_opts = pn.Card(pn.Column(usable_indices,
                                      enable_search,
                                      w_density_factor,
                                      info,
                                      w_show_poor), collapsed=True, 
                            title={"en":"Advanced options","fr":"Options avancées"}[language],
                           width=300,max_width=300,min_width=300)
    
    sidebar.clear()
    sidebar.append(w_sidetitle)
    sidebar.append(w_city)
    sidebar.append(w_ssp_labelled)
    sidebar.append(w_tgt_period)
    sidebar.append(w_indices)
    sidebar.append(advanced_opts)
    sidebar.append(w_run)
    sidebar.append(w_progress)
    @pn.depends(clicks=w_run.param.clicks, watch=True)
    def t_run_search(clicks):
        pane = analogs_search(clicks)

        searches.append(pane)
        searches.active = len(searches.objects) - 1
    

def change_language(event):
    global LOCALE
    LOCALE = "fr" if (LOCALE == "en") else "en"
    w_language.name = "English" if LOCALE == "fr" else "Français"
    w_sidetitle.object = {"en":"##Start new search","fr":"##Débutez nouvelle recherche"}[LOCALE]
    searches.clear()
    searches.append(get_helppage(LOCALE))
    searches.closablelist[0] = False
    update_handled(language=LOCALE)
    
w_language.on_click(change_language)

def close_modal_set_english(event):
    global LOCALE
    dash.close_modal()
    if LOCALE == 'fr':
        change_language(event)

def close_modal_set_french(event):
    global LOCALE
    dash.close_modal()
    if LOCALE == 'en':
        change_language(event)

w_enter_en.on_click(close_modal_set_english)
w_enter_fr.on_click(close_modal_set_french)

In [ ]:

pn.state.onload(update_handled)
t4 = time.time()
print('time to full load:', t4 - t1)

To use this dashboard from within PAVICS and have it run in your user account use the first line of the next cell (`s = dash.show(...)`) and comment the second one (`dash.servable()`). In that case, if you want to update the dashboard after making changes, don't forget to run `s.stop()` before rerunning  `s = dash.show(...)`.

In [ ]:
# s = dash.show(port=9093, websocket_origin='*')
dash.servable()
# print(f"The line above is lying to you. The _real_ adress is:\n https://pavics.ouranos.ca/jupyter/user-redirect/proxy/{s.port}/")

In [ ]:
# s.stop()
